In [ ]:
import atomap.api as am
import numpy as np
import hyperspy.api as hs
import atomap.dummy_data as dummy_data

s = hs.load('/Users/usuario/Python PM Office/20231012/DF Stack_04 aligned sum Try ABSF filtered STEM filtered Inverse.dm3') #introduce first inverse ABF or HAADF
s.plot()

In [ ]:
%matplotlib nbagg
A_positions = am.get_atom_positions(s, separation=19)
sublattice_A = am.Sublattice(A_positions, image=s.data, color='r')
atom_positions_new = am.add_atoms_with_gui(s, A_positions)

In [ ]:
atom_positions_select = am.select_atoms_with_gui(s, atom_positions_add)

In [ ]:
atom_positions_new = am.add_atoms_with_gui(s, A_positions)

In [ ]:
sublattice_A = am.Sublattice(atom_positions_new, s)
sublattice_A.find_nearest_neighbors()
sublattice_A.refine_atom_positions_using_center_of_mass()
sublattice_A.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3)
sublattice_A.construct_zone_axes(atom_plane_tolerance=0.6)
sublattice_A.plot()
sublattice_A.plot_planes()

In [ ]:
zone_axis_001 = sublattice_A.zones_axis_average_distances[2] #1 introduces vertical zone axes
B_positions = sublattice_A.find_missing_atoms_from_zone_vector(zone_axis_001)

In [ ]:
from atomap.tools import remove_atoms_from_image_using_2d_gaussian
image_without_A = remove_atoms_from_image_using_2d_gaussian(sublattice_A.image, sublattice_A)

In [ ]:
sublattice_B = am.Sublattice(B_positions, image_without_A, color='blue')
sublattice_B.construct_zone_axes()
sublattice_B.refine_atom_positions_using_center_of_mass()
sublattice_B.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3)
sublattice_B.plot_planes()
atom_lattice = am.Atom_Lattice(image=s.data, name='test', sublattice_list=[sublattice_A, sublattice_B])
atom_lattice.plot()

In [ ]:
s_ABF = hs.load('/Users/usuario/Python PM Office/UCL/20230922/L_c23012_01/DF Stack_06 aligned sum Wiener Filtered RL STEM.dm3')
s_ABF.plot()

In [ ]:
initial_positions = sublattice_A.atom_positions
sublattice_A2 = am.Sublattice(initial_positions, image=np.divide(1, s_ABF.data), color='r') #if the image is already inversed use  s_ABF.data without image=np.devide 
atom_positions_add = am.add_atoms_with_gui(np.divide(1, s_ABF.data), initial_positions)

In [ ]:
sublattice_A2 = am.Sublattice(atom_positions_add, image=np.divide(1, s_ABF.data), color='r')
sublattice_A2.find_nearest_neighbors()
sublattice_A2.refine_atom_positions_using_center_of_mass()
sublattice_A2.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3)
sublattice_A2.construct_zone_axes()
image_without_A2 = remove_atoms_from_image_using_2d_gaussian(sublattice_A2.image, sublattice_A2)

In [ ]:
initial_positions = sublattice_B.atom_positions
sublattice_B2 = am.Sublattice(initial_positions, image=image_without_A2, color='b')
atom_positions_new = am.add_atoms_with_gui(s_ABF, initial_positions)

In [ ]:
sublattice_B2 = am.Sublattice(atom_positions_new, image=image_without_A2, pixel_size=4, color='b')
sublattice_B2.find_nearest_neighbors()
sublattice_B2.refine_atom_positions_using_center_of_mass() 
sublattice_B2.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3)#It is better not using them when the image is blur or use smaller percent
sublattice_B2.construct_zone_axes()
sublattice_B2.plot_planes()
sublattice_B2.plot()

In [ ]:
zone_axis_002 = sublattice_B2.zones_axis_average_distances[0]# 0 since the oxygen is parallel to B2
O_positions = sublattice_B2.find_missing_atoms_from_zone_vector(zone_axis_002,vector_fraction=0.5,extend_outer_edges=True, outer_edge_limit=5)
image_without_AB = remove_atoms_from_image_using_2d_gaussian(sublattice_B2.image, sublattice_B2)

In [ ]:
sublattice_O = am.Sublattice(O_positions, image_without_AB, color='g') #or use image_without_AB instead of s. but preferably use s
atom_positions_new = am.add_atoms_with_gui(s_ABF, O_positions) #instead of s I can use image without_AB however sometimes the quality of this image is bad

In [ ]:
sublattice_O = am.Sublattice(O_positions, image_without_AB, color='g') #or use s instead of image_without_A. but preferably use image without A
sublattice_O.construct_zone_axes()
sublattice_O.refine_atom_positions_using_center_of_mass()# for oxygen is better to use both fitting methods
sublattice_O.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3) # the % of the gaussia radious. 

In [ ]:
atom_lattice = am.Atom_Lattice(image=s_ABF.data, name='ABO3', sublattice_list=[sublattice_A, sublattice_B, sublattice_O])
atom_lattice.plot()
atom_lattice.plot(image=s.data)

In [ ]:
import temul.api as tml
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_O = atom_lattice.sublattice_list[2]
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[2] # change this depending on the types of oxygen. (use either [1] and [2] or [0] and [2]
za1 = sublattice_A.zones_axis_average_distances[0]
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za1, sublattice_O)
w=vector_list = s_p.metadata.vector_list

# function to return the second element of the
def colsort():
    def sortSecond(val):
        return val[1] 
  
        #sort each column individualy. 
        # using the second key 
    
    rows=15 #number of Oxygen rows.Those number are not the same as above in the case of 001 ABF images, missing 2 columns in the case of O1 
    columns=22  #number of Oxygen columns                                                                missing 1 rows adding 1 column in the case of O2
    
    d=rows
    p=0
    w.sort() #sort the list to the first element of each item which is the row.
    for i in range(columns): #number of columns
        s=w[p:d] #Select all the elemnts of the first column
        s.sort(key = sortSecond) #Sort the element from the top to bottom of the column
        w[p:d]=s # replace the elements of the list with the sorted ones
        p=p+rows
        d=d+rows
colsort()
w

#print(len(u)) #check the length of the vector so as to create a list with the same size

In [ ]:
#**********Dipole filter**********
colsort()
x, y = [i[0] for i in w], [i[1] for i in w]
u, v = [i[2] for i in w], [i[3] for i in w]

elements=330 # number of Oxygen atoms
rows=15 # bumber of Oxygen rows
resolution=0.008902054

#the list below is used to interate all over the vectors in the u and v lists.
list_n = [*range(elements)]

#bottom multiplication
w.append(w.pop(0)) # puts the first value of the list at the end of it. This process used so as to multiply the first vector with the one below.
u = [0.3*u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [0.3*v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#up multiplication
colsort()
for b in range(elements-1):
    w.append(w.pop(0))#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#side +1 column
colsort()
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.15*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.15*i[3] for i,t in zip(w,list_n)]

#side +2 column
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#side -1 column
colsort()
for b in range(elements-rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.15*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.15*i[3] for i,t in zip(w,list_n)]

#side -2 column
colsort()
for b in range(elements-rows*2):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#print(w)  #use this function to check if the columns are sepparated correctly
#for diagonal multiplication a combination between bottom and side multiplication is needed. Specifically "for b in range(n+1)"

sampling, units =  resolution, 'nm' #image calibration
tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,sampling=sampling, units=units,unit_vector=False, save=None, scalebar=True, plot_style='vector', color='r', overlay=True, monitor_dpi=200)


In [ ]:
#polarization map
import temul.api as tml
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_O = atom_lattice.sublattice_list[2]
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[2]
za1 = sublattice_A.zones_axis_average_distances[0]
sublattice_A.plot_planes(image=atom_lattice.image)# use this function to reveal the planes and select the appropriate ones. The number of the plane is indicated on the top
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za1, sublattice_O)
vector_list = s_p.metadata.vector_list
x, y = [i[0] for i in vector_list], [i[1] for i in vector_list]
u, v = [i[2] for i in vector_list], [i[3] for i in vector_list]

sampling, units = 0.008902054, 'nm' #image calibration
tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,sampling=sampling, units=units,unit_vector=False, save=None, scalebar=True, plot_style='vector', color='r', overlay=True, monitor_dpi=150)

In [ ]:
# filtered averaging
import statistics
import numpy
def mean(data):
    """Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss
def stddev(data, ddof=0):
    """Calculates the population standard deviation
    by default; specify ddof=1 to compute the sample
    standard deviation."""
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

#Here I use the filter to get better averages values
colsort()        

x, y = [i[0] for i in w], [i[1] for i in w]
u, v = [i[2] for i in w], [i[3] for i in w]

#print(len(u)) #check the length of the vector so as to create a list with the same size

elements=196 # number of Oxygen atoms
rows=14 # bumber of Oxygen rows
columns=14  #number of Oxygen columns
resolution=0.006139348

#the list below is used to interate all over the vectors in the u and v lists.
list_n = [*range(elements)]

#bottom multiplication
w.append(w.pop(0)) # puts the first value of the list at the end of it. This process used so as to multiply the first vector with the one below.
u = [0.3*u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [0.3*v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#up multiplication
colsort()
for b in range(elements-1):
    w.append(w.pop(0))#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#side +1 column
colsort()
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.15*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.15*i[3] for i,t in zip(w,list_n)]

#side +2 column
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#side -1 column
colsort()
for b in range(elements-rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.15*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.15*i[3] for i,t in zip(w,list_n)]

#side -2 column
colsort()
for b in range(elements-rows*2):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#End of the filter



g=0;
c=0;
stdvp=list=[]  #list for stdv of horizontal displacement
stdvq=list=[]  #list for stdv of vertical displacement


print("        Δx                 Δy               STDVx              STDVy")
for q in range(rows): #this range is the number of rows
    t=q
    for i in range(columns): #this range is the number of columns
        b=u[t]
        z=v[t]
        c=c+b
        g=g+z
        t=t+rows     #this range is the number of rows
        stdvp.append(b*1000* resolution)
        stdvq.append(z*1000* resolution)
    
       # stdvq.append(z)
   ##  STDVp=numpy.std(arrp, axis=0)
   # arrq = numpy.array([q])
  #  e=numpy.std(arrq, axis=0)
    STDVp=stddev(stdvp)
    STDVq=stddev(stdvq)

    l=(1000*c*resolution)/columns  #horizontal x average value in pm
    a=(1000*g*resolution)/columns  #and vertical y average value in pm
    print(l, a, STDVp , STDVq ) 
    c=0;
    g=0;      
    stdvp=list=[]  #list for stdv of horizontal displacement
    stdvq=list=[]  #list for stdv of horizontal displacement
t=0    